# Results
In this notebook, we define and calculate the statistics of our tests.

In [1]:
# load data

import json
import pandas as pd
from collections import defaultdict
import math
import os

def loadData(fileName):
    '''Loading data from result files'''
    with open(f'{fileName}.json', 'r', encoding='utf-8') as file:
        data = json.load(file)
        return data

def normalize(cases):
    '''make all the ddx list of a case of the same length by padding with None'''
    for case in cases.values():
        maxLen = max(len(result) for result in case.values())
        for result in case.values():
            result += [None]*(maxLen-len(result))

        assert len(set(len(result) for result in case.values())) == 1
    
    return cases

def getDataframe(case):
    '''Convert each test case into a dataframe'''
    return pd.DataFrame(
        case,
        columns=['gs',*sorted([key for key in case.keys() if key != 'gs'])],
        index= list(range(1,1+len(list(case.values())[0])))
        )


In [2]:
# We need to make all the differentials of the same length to ease comparison
# We pad the lists with None
data = loadData('allResults')
keys = loadData('Dr. Maram 2nd batch 60 plus 7').keys()
data = {k: v for k, v in data.items() if k in [
    '75', '64', '63', '62', '60', '59', '58', '53', '52', '24', '17','16', '13', '11', '6', '4', '2'] + list(keys)}
for caseNum in data:
    for app in data[caseNum]:
        data[caseNum][app] = [r for r in data[caseNum][app] if r]
for caseNum, tests in data.items():
    assert len(tests.keys()) == 10, f"app missing in case {caseNum}, {tests.keys()}"
normalizedData = normalize(data)
cases = {int(id): getDataframe(case) for id, case in normalizedData.items()}
caseClassification = loadData('case-classification')
f'We have {len(cases)} cases in the experiment.'


'We have 77 cases in the experiment.'

Let us have a look at all the cases.

In [3]:
from IPython.display import display
for caseNum, case in cases.items():
    print(f"Case number {caseNum}")
    display(case)

Case number 2


,gs,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,WebMD
1,unstable angina,unstable angina,unstable angina,None,None,myocardial infarction,stable angina,unstable angina,myocardial infarction,myocardial infarction
2,myocardial infarction,collapsed lung,myocardial infarction,None,None,musculoskeletal chest pain,unstable angina,myocardial infarction,unstable angina,broken ribs
3,stable angina,myocardial infarction,stable angina,None,None,cardia arrhythmias,myocardial infarction,None,heart failure,costochondritis
4,pericardial effusion,chronic obstructive pulmonary disease,varient angina,None,None,unstable angina,None,None,None,heartburn/gerd
5,None,stable angina,None,None,None,None,None,None,None,unstable angina
6,None,None,None,None,None,None,None,None,None,pulmonary embolism


Case number 4


,gs,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,WebMD
1,asthma,chronic obstructive pulmonary disease,asthma,None,heart issue,asthma,asthma,asthma,asthma,asthma
2,chronic obstructive pulmonary disease,asthma,chronic obstructive pulmonary disease,None,asthma,gastroesophageal reflux,acute bronchitis,pulmonary edema,pulmonary fibrosis,chronic obstructive pulmonary disease
3,None,chronic sarcoidosis,cystic fibrosis,None,chronic obstructive pulmonary disease,heart failure,hay fever,None,sarcoidosis,pulmonary embolism
4,None,paradoxical vocal fold motion,bronchiactasis,None,None,None,None,None,None,pneumonia
5,None,None,valvular heart disease,None,None,None,None,None,None,bronchitis
6,None,None,acute bronchitis,None,None,None,None,None,None,diastolic heart failure


Case number 6


...

,gs,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,WebMD
1,sickle cell anemia,g6pd,septic arthritis,None,None,None,mononucleosis,sickle cell anemia,sickle cell anemia,pneumonia
2,idiopathic juvenile arthritis,acute cholangitis,leukemia,None,None,None,sickle cell anemia,None,None,subacute bacterial endocarditis
3,None,leptospirosis,bronchiolitis,None,None,None,leukemia,None,None,influenza
4,None,gilbert's syndrome,malaria,None,None,None,None,None,None,lymphocytic lymphoma
5,None,sickle cell anemia,influenza,None,None,None,None,None,None,septicemia
6,None,None,idiopathic juvenile arthritis,None,None,None,None,None,None,mononucleosis


Case number 300


,gs,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,WebMD
1,uterine fibroids,uterine fibroids,secondary dysmenorrhea,None,uterine fibroids,None,uterine fibroids,uterine fibroids,uterine fibroids,acute appendicitis
2,adenomyosis,adenomyosis,adenomyosis,None,None,None,endometrial polyps,adenomyosis,adenomyosis,irritable bowel syndrome
3,None,endometrial polyps,uterine fibroids,None,None,None,uterine cancer,None,None,prostaitis
4,None,endometriosis,endometrial cancer,None,None,None,None,None,None,interstitial cystitis
5,None,menopause transition,None,None,None,None,None,None,None,myocardial infarction
6,None,None,None,None,None,None,None,None,None,heart failure


Case number 302


,gs,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,WebMD
1,dengue fever,viral sinusitis,malaria,None,coronavirus disease 2019,influenza,coronavirus disease 2019,hay fever,dengue fever,pneumococcal meningitis
2,malaria,acute appendicitis,influenza,None,Influenza or flu-like illness,lyme disease,malaria,None,rocky mountain spotted fever,influenza
3,None,herpes infection in the mouth,dengue fever,None,giant cell arteritis,None,hepatitis,None,None,acute appendicitis
4,None,tonsillitis,osteomyelitis,None,None,None,lishmaniasis,None,None,septicemia
5,None,osteomyelitis,anemia,None,None,None,None,None,None,pneumonia
6,None,None,infective endocarditis,None,None,None,None,None,None,kidney infection (pyelonephritis)


## Let us define the metrics now.

### Terms used
- TP: True positive (correct disease retrieved)
- TN: True negative (wrong disease **not** retrieved)
- FP: False positive (wrong disease retrieved)
- FN: False negative (correct disease **not** retrieved)
- gold standard - the correct list of diseases as determined by collective intelligence of doctors

### Precision
Precision helps us understand how exact our results are. It gives us an intuition about how many wrong diseases (false positives) are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the complete list retrieved*.
$$precision = \frac{TP}{TP + FP} = \frac{TP}{\text{length of differential list}}$$

### Recall
Recall is a measure of how many of the correct diseases are being retrieved. It is the ratio *number of correct diseases retrieved* to the *length of the gold standard list*.
$$recall = \frac{TP}{TP + FN} = \frac{TP}{\text{length of the gold standard}}$$

### F1 Score
F1 score is the weighted harmonic mean of *precision* and *recall*. It is a metric that combines *precision* and *recall* and gives us 1 score for easier comparison.

Suppose $\beta$ defines how important is $recall$ to $precision$ then,
$$fscore_{\beta} = (1 + \beta^2)\frac{precision \cdot recall}{(\beta^2 \cdot precision) + recall}$$
Substituting $\beta = 1$,
$$fscore_{1} = \frac{2 \cdot precision \cdot recall}{ precision + recall}$$

### NDCG
NDCG or Normalized Discounted Cumulative Gain is measure of how accurate the ranking is. In our calculations, we use
$$DCG = \sum_{i=1}^n\frac{2^{relevance_i}-1}{log_2(i+1)}$$
where $n$ is the number of differentials in the returned list and  
$relevance_i = |gold standard| - rank_{gold\ standard}(ddx[i])$ if $ddx[i]$ is present, 0 otherwise.

$$NDCG = \frac{DCG_{ddx}}{DCG_{gold\ standard}}$$

### M Score
M Score determines where the top disease (gold standard) appears in the returned differential.
$$M_i = \text{gold standard[0]} \in \text{ddx[:i]}$$

### Position
Shows the position of the gold standard[0] in the returned differential. 

### Length
$$length = \frac{|ddx|}{|gold\ standard|}$$


In [4]:
import math

beta = 1 #used in recall

def getPrecision(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp if tp == 0 else tp/candidate.count()


def getRecall(goldStandard: pd.Series, candidate: pd.Series) -> float:
    tp = sum(int(disease in goldStandard.values and disease is not None)
             for disease in candidate)
    return tp/goldStandard.count()


def getF1Score(precision: float, recall: float, beta: float = 1) -> float:
    return math.nan if precision+recall == 0 else \
        (1+beta**2)*precision*recall/(precision*(beta**2)+recall)


def getNDCG(goldStandard: pd.Series, candidate: pd.Series, scores) -> float:
    def discount(score: float, index: int) -> float:
        '''The index is 1 based'''
        return (math.pow(2, score)-1)/math.log2(index+1)

    maxDCG = sum(discount(scores[i], i+1) for i in range(len(scores)))

    candidateRelevance = []
    goldStandard = list(goldStandard)
    for index, disease in enumerate(candidate):
        if disease is not None and disease in goldStandard:
            candidateRelevance.append(
                discount(scores[goldStandard.index(disease)], index+1))
        else:
            candidateRelevance.append(0)

    return sum(candidateRelevance)/maxDCG


def getMScore(goldStandard: pd.Series, candidate: pd.Series, m=1) -> bool:
    return goldStandard.values[0] in candidate.values[:m]


def getPosition(goldStandard: pd.Series, candidate: pd.Series) -> bool:
    return math.nan if goldStandard.values[0] not in candidate.values else\
        1 + list(candidate.values).index(goldStandard.values[0])


def getLength(goldStandard: pd.Series, candidate: pd.Series) -> int:
    return math.nan if candidate.count() == 0 else \
        candidate.count()/goldStandard.count()


def getScoresCase(case: pd.DataFrame, beta:float = 1) -> pd.DataFrame:
    scores = [
        [getPrecision(case.iloc[:, 0], case.iloc[:, i])
         for i in range(1, len(case.columns))],
        [getRecall(case.iloc[:, 0], case.iloc[:, i])
         for i in range(1, len(case.columns))]
    ]

    scores.append([getF1Score(scores[0][i], scores[1][i],beta=beta)
                  for i in range(len(case.columns)-1)])
    
    # relevance for a list of 4 is 4, 3, 2, 1
    # relevance for a list of 2 is 2, 1
    scores.append([getNDCG(case.iloc[:, 0], case.iloc[:, i],
                           list(range(case.iloc[:, 0].count(), 0, -1)))
                   for i in range(1, len(case.columns))])

    for m in range(1, 6, 2):
        scores.append([getMScore(case.iloc[:, 0], case.iloc[:, i], m)
                      for i in range(1, len(case.columns))])

    scores.append([getPosition(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    scores.append([getLength(case.iloc[:, 0], case.iloc[:, i])
                   for i in range(1, len(case.columns))])

    return pd.DataFrame(scores, columns=case.columns[1:],
                        index=["precision", "recall", "f1-score", "NDCG",
                               "M1", "M3", "M5", "position", "length (x of gs)"],
                        )

scores = {id:getScoresCase(case) for id, case in cases.items()}

In [5]:
scores[2]
def rankScore(score):
    return score.loc['M1','Avey']

rankedScores = sorted([(k,rankScore(v)) for k,v in scores.items()],key=lambda x: x[1])
# print([a for a,b in rankedScores])
display(dict(rankedScores))

{13: False,
 16: False,
 17: False,
 24: False,
 52: False,
 53: False,
 60: False,
 62: False,
 64: False,
 75: False,
 84: False,
 85: False,
 86: False,
 91: False,
 104: False,
 125: False,
 126: False,
 144: False,
 154: False,
 156: False,
 159: False,
 164: False,
 169: False,
 171: False,
 174: False,
 178: False,
 179: False,
 180: False,
 205: False,
 207: False,
 228: False,
 254: False,
 258: False,
 260: False,
 267: False,
 279: False,
 281: False,
 284: False,
 298: False,
 300: False,
 302: False,
 2: True,
 4: True,
 6: True,
 11: True,
 58: True,
 59: True,
 63: True,
 82: True,
 87: True,
 88: True,
 89: True,
 92: True,
 122: True,
 132: True,
 152: True,
 155: True,
 161: True,
 162: True,
 165: True,
 170: True,
 187: True,
 188: True,
 204: True,
 206: True,
 208: True,
 209: True,
 213: True,
 230: True,
 231: True,
 250: True,
 252: True,
 256: True,
 257: True,
 277: True,
 278: True,
 283: True}

Let us define the experiment now. We will pick which cases to compute startistics for.

In [6]:
from collections import defaultdict
experiments = {}
combineLabels = defaultdict(list)

def addExperiment(caseType,casesToConsider):
    #add experiment to ignore no case
    experiments[f'ignore_{caseType}_none'] = set(casesToConsider)

    for app, classifications in caseClassification['apps'].items():
        for classification, caseNums in classifications.items():
            #record labels to combine later
            combineLabels[f'ignore_{caseType}_{classification}'].append(f'ignore_{caseType}_{app}_{classification}')
            #experiment to ignore the cases for the particular app
            experiments[f'ignore_{caseType}_{app}_{classification}'] = set(casesToConsider) - set(caseNums)
            #experiment to ignore the cases for all the apps
            if f'ignore_{caseType}_any_{classification}' in experiments:
                experiments[f'ignore_{caseType}_any_{classification}'] =\
                 experiments[f'ignore_{caseType}_any_{classification}'] - set(caseNums)
            else:
                experiments[f'ignore_{caseType}_any_{classification}'] = experiments[f'ignore_{caseType}_{app}_{classification}']

addExperiment('common',caseClassification['common'])
addExperiment('less-common',caseClassification['less common'])
addExperiment('all-cases',set(caseClassification['less common']) | set(caseClassification['common']))
    

list(enumerate(experiments.keys()))


[(0, 'ignore_common_none'),
 (1, 'ignore_common_Ada_session_failed'),
 (2, 'ignore_common_any_session_failed'),
 (3, 'ignore_common_Ada_no_disease_found'),
 (4, 'ignore_common_any_no_disease_found'),
 (5, 'ignore_common_Avey_session_failed'),
 (6, 'ignore_common_Avey_no_disease_found'),
 (7, 'ignore_common_Babylon_session_failed'),
 (8, 'ignore_common_Babylon_no_disease_found'),
 (9, 'ignore_common_Buoy_session_failed'),
 (10, 'ignore_common_Buoy_no_disease_found'),
 (11, 'ignore_common_K health_session_failed'),
 (12, 'ignore_common_K health_no_disease_found'),
 (13, 'ignore_common_WebMD_session_failed'),
 (14, 'ignore_common_WebMD_no_disease_found'),
 (15, 'ignore_less-common_none'),
 (16, 'ignore_less-common_Ada_session_failed'),
 (17, 'ignore_less-common_any_session_failed'),
 (18, 'ignore_less-common_Ada_no_disease_found'),
 (19, 'ignore_less-common_any_no_disease_found'),
 (20, 'ignore_less-common_Avey_session_failed'),
 (21, 'ignore_less-common_Avey_no_disease_found'),
 (22, 'ig

In [7]:
def getAverage(scores, row: int, col: int) -> float:
    values = []
    for score in scores.values():
        if not math.isnan(score.iloc[row, col]):
            values.append(score.iloc[row, col])

    return 0 if not values else round(sum(values)/len(values), 3)

results = {}
for label, casesToConsider in experiments.items():
    selectedScores = {id:score for id,score in scores.items() if int(id) in casesToConsider}
    columns = next(iter(scores.values())).columns
    averageScores = pd.DataFrame(
        [
            [
                getAverage(selectedScores, row, col)
                for col in range(len(columns))
            ]
            for row in range(9)
        ],
        columns=columns,
        index=[
            f"average_{x}" for x in [
                "precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length (x of gs)"
            ]
        ]
    )

    # ignore nan for recall
    for col in next(iter(scores.values())).columns:
        p = averageScores.loc["average_precision", col]
        r = averageScores.loc["average_recall", col]
        averageScores.loc["average_f1-score",
                        col] = round(getF1Score(p, r, beta), 3)

    doctorResults = averageScores.loc[:, "MA"] +\
        averageScores.loc[:, "NJ"] +\
        averageScores.loc[:,
                            "TH"]
    doctorResults /= 3.0
    
    averageScores.insert(
        loc=8, column="average_doctor",
        value=doctorResults.round(3),
    )

    results[label] = averageScores


Let us print all the results. The experiments are nomenclatured as follows:
- **[common|uncommon|all]:** means whether only common cases were considered, only uncommon cases were considered, and so on.
- **app name / any:** If an app name is present, then we ignore only those cases that pertain to it. If the label is *any* then we consider all apps in that experiment.
- **failure type**: The apps can fail in 2 ways. Either a session does not complete due to some reason or the app fails to retrieve any diagnosis. If this is set to None, then we ignore the failures and consider all cases under option 1 above.

In [8]:
def displayResults(results,printNumCases=True):
    for label, result in results.items():
        result.to_csv(f'stats/{label}.json',sep=';')
        if printNumCases:
            print(f'Results for experiment {label}, which has {len(set(experiments[label]) & set(scores.keys()))} cases, is')
        else:
            print(f'Results for experiment {label} is')
        display(result)

displayResults({key:val for key, val in results.items() if 'any' not in key})

Results for experiment ignore_common_none, which has 49 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.482,0.389,0.022,0.320,0.461,0.589,0.830,0.779,0.733,0.252
average_recall,0.620,0.663,0.024,0.319,0.467,0.444,0.385,0.574,0.468,0.559
average_f1-score,0.542,0.490,0.023,0.319,0.464,0.506,0.526,0.661,0.564,0.347
average_NDCG,0.721,0.664,0.026,0.369,0.598,0.588,0.619,0.775,0.661,0.488
average_M1,0.612,0.469,0.020,0.306,0.571,0.551,0.694,0.776,0.674,0.265
average_M3,0.878,0.816,0.020,0.490,0.673,0.735,0.735,0.959,0.810,0.469
average_M5,0.898,0.837,0.041,0.490,0.714,0.735,0.735,0.959,0.810,0.612
average_position,1.432,1.810,2.500,1.542,1.371,1.306,1.056,1.213,1.192,2.765
average_length (x of gs),1.554,1.901,0.967,1.020,1.227,0.889,0.520,0.825,0.745,2.523


Results for experiment ignore_common_Ada_session_failed, which has 49 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.482,0.389,0.022,0.320,0.461,0.589,0.830,0.779,0.733,0.252
average_recall,0.620,0.663,0.024,0.319,0.467,0.444,0.385,0.574,0.468,0.559
average_f1-score,0.542,0.490,0.023,0.319,0.464,0.506,0.526,0.661,0.564,0.347
average_NDCG,0.721,0.664,0.026,0.369,0.598,0.588,0.619,0.775,0.661,0.488
average_M1,0.612,0.469,0.020,0.306,0.571,0.551,0.694,0.776,0.674,0.265
average_M3,0.878,0.816,0.020,0.490,0.673,0.735,0.735,0.959,0.810,0.469
average_M5,0.898,0.837,0.041,0.490,0.714,0.735,0.735,0.959,0.810,0.612
average_position,1.432,1.810,2.500,1.542,1.371,1.306,1.056,1.213,1.192,2.765
average_length (x of gs),1.554,1.901,0.967,1.020,1.227,0.889,0.520,0.825,0.745,2.523


Results for experiment ignore_common_Ada_no_disease_found, which has 49 cases, is


...

,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.443,0.389,0.014,0.260,0.374,0.567,0.807,0.738,0.704,0.218
average_recall,0.567,0.677,0.016,0.271,0.411,0.427,0.384,0.595,0.469,0.507
average_f1-score,0.497,0.494,0.015,0.265,0.392,0.487,0.520,0.659,0.555,0.305
average_NDCG,0.628,0.652,0.016,0.285,0.463,0.521,0.567,0.760,0.616,0.431
average_M1,0.506,0.468,0.013,0.208,0.403,0.455,0.610,0.753,0.606,0.234
average_M3,0.753,0.779,0.013,0.377,0.506,0.649,0.636,0.922,0.736,0.390
average_M5,0.805,0.818,0.026,0.377,0.532,0.649,0.636,0.922,0.736,0.532
average_position,1.629,1.812,2.500,1.621,1.463,1.380,1.041,1.239,1.220,2.667
average_length (x of gs),1.618,2.031,0.967,1.094,1.270,0.911,0.512,0.918,0.780,2.615


Results for experiment ignore_all-cases_WebMD_session_failed, which has 76 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.445,0.388,0.014,0.263,0.373,0.561,0.805,0.735,0.700,0.221
average_recall,0.568,0.673,0.016,0.275,0.410,0.426,0.383,0.596,0.468,0.514
average_f1-score,0.499,0.492,0.015,0.269,0.391,0.484,0.519,0.658,0.554,0.309
average_NDCG,0.625,0.648,0.017,0.289,0.458,0.517,0.563,0.759,0.613,0.436
average_M1,0.500,0.461,0.013,0.211,0.395,0.447,0.605,0.750,0.601,0.237
average_M3,0.750,0.776,0.013,0.382,0.500,0.645,0.632,0.921,0.733,0.395
average_M5,0.803,0.816,0.026,0.382,0.526,0.645,0.632,0.921,0.733,0.539
average_position,1.639,1.825,2.500,1.621,1.475,1.388,1.042,1.243,1.224,2.667
average_length (x of gs),1.613,2.025,0.967,1.086,1.274,0.916,0.512,0.923,0.784,2.615


Results for experiment ignore_all-cases_WebMD_no_disease_found, which has 76 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.446,0.389,0.014,0.259,0.371,0.561,0.805,0.735,0.700,0.221
average_recall,0.568,0.679,0.016,0.268,0.403,0.426,0.383,0.596,0.468,0.514
average_f1-score,0.500,0.495,0.015,0.263,0.386,0.484,0.519,0.658,0.554,0.309
average_NDCG,0.625,0.650,0.017,0.278,0.456,0.517,0.563,0.759,0.613,0.436
average_M1,0.500,0.461,0.013,0.197,0.395,0.447,0.605,0.750,0.601,0.237
average_M3,0.750,0.776,0.013,0.368,0.500,0.645,0.632,0.921,0.733,0.395
average_M5,0.803,0.816,0.026,0.368,0.526,0.645,0.632,0.921,0.733,0.539
average_position,1.639,1.825,2.500,1.643,1.475,1.388,1.042,1.243,1.224,2.667
average_length (x of gs),1.606,2.038,0.967,1.086,1.266,0.916,0.512,0.923,0.784,2.615


In [9]:
displayResults({key:val for key, val in results.items() if 'any' in key})

Results for experiment ignore_common_any_session_failed, which has 40 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.501,0.367,0.027,0.329,0.490,0.585,0.817,0.729,0.710,0.278
average_recall,0.618,0.628,0.030,0.332,0.485,0.431,0.371,0.583,0.462,0.585
average_f1-score,0.553,0.463,0.028,0.330,0.487,0.496,0.510,0.648,0.551,0.377
average_NDCG,0.706,0.637,0.031,0.391,0.637,0.563,0.594,0.760,0.639,0.523
average_M1,0.575,0.425,0.025,0.325,0.600,0.500,0.650,0.725,0.625,0.300
average_M3,0.850,0.800,0.025,0.525,0.725,0.725,0.700,0.950,0.792,0.500
average_M5,0.875,0.825,0.050,0.525,0.775,0.725,0.700,0.950,0.792,0.650
average_position,1.486,1.912,2.500,1.571,1.419,1.379,1.071,1.263,1.238,2.724
average_length (x of gs),1.529,1.908,0.967,1.024,1.225,0.885,0.525,0.890,0.767,2.417


Results for experiment ignore_common_any_no_disease_found, which has 4 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.792,0.458,0.271,0.208,0.625,0.417,1.000,0.833,0.750,0.143
average_recall,0.642,0.550,0.300,0.175,0.433,0.392,0.442,0.508,0.447,0.275
average_f1-score,0.709,0.500,0.285,0.190,0.512,0.404,0.613,0.631,0.549,0.188
average_NDCG,0.774,0.696,0.314,0.300,0.630,0.342,0.814,0.850,0.669,0.196
average_M1,0.750,0.500,0.250,0.250,0.750,0.250,1.000,1.000,0.750,0.000
average_M3,1.000,1.000,0.250,0.500,0.750,0.250,1.000,1.000,0.750,0.000
average_M5,1.000,1.000,0.500,0.500,0.750,0.250,1.000,1.000,0.750,0.000
average_position,1.250,1.500,2.500,1.500,1.000,1.000,1.000,1.000,1.000,7.000
average_length (x of gs),0.850,1.633,0.967,0.808,0.689,0.950,0.442,0.608,0.667,1.975


Results for experiment ignore_less-common_any_session_failed, which has 19 cases, is


...

,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.700,1.000,0.0,0.000,0.0,0.500,1.000,0.750,0.750,0.100
average_recall,0.583,0.417,0.0,0.000,0.0,0.250,0.417,0.417,0.361,0.250
average_f1-score,0.636,0.589,NaN,NaN,NaN,0.333,0.589,0.536,0.486,0.143
average_NDCG,0.848,0.786,0.0,0.000,0.0,0.138,0.573,0.786,0.499,0.178
average_M1,1.000,1.000,0.0,0.000,0.0,0.000,0.500,1.000,0.500,0.000
average_M3,1.000,1.000,0.0,0.000,0.0,0.000,0.500,1.000,0.500,0.000
average_M5,1.000,1.000,0.0,0.000,0.0,0.000,0.500,1.000,0.500,0.500
average_position,1.000,1.000,0.0,0.000,0.0,0.000,1.000,1.000,0.667,4.000
average_length (x of gs),1.083,0.417,0.0,0.667,0.0,0.750,0.417,0.583,0.583,2.250


Results for experiment ignore_all-cases_any_session_failed, which has 59 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.478,0.362,0.018,0.280,0.438,0.568,0.782,0.709,0.686,0.252
average_recall,0.590,0.660,0.020,0.297,0.477,0.414,0.369,0.604,0.462,0.560
average_f1-score,0.528,0.468,0.019,0.288,0.457,0.479,0.501,0.652,0.544,0.348
average_NDCG,0.641,0.633,0.021,0.324,0.539,0.505,0.525,0.745,0.592,0.480
average_M1,0.492,0.424,0.017,0.237,0.458,0.441,0.542,0.712,0.565,0.271
average_M3,0.780,0.763,0.017,0.441,0.593,0.627,0.576,0.898,0.700,0.441
average_M5,0.814,0.814,0.034,0.441,0.627,0.627,0.576,0.898,0.700,0.576
average_position,1.604,1.939,2.500,1.654,1.514,1.351,1.059,1.245,1.218,2.595
average_length (x of gs),1.556,2.038,0.967,1.101,1.268,0.873,0.518,0.944,0.778,2.522


Results for experiment ignore_all-cases_any_no_disease_found, which has 6 cases, is


,Ada,Avey,Babylon,Buoy,K health,MA,NJ,TH,average_doctor,WebMD
average_precision,0.761,0.639,0.181,0.139,0.417,0.444,1.000,0.806,0.750,0.129
average_recall,0.622,0.506,0.200,0.117,0.289,0.344,0.433,0.478,0.418,0.267
average_f1-score,0.685,0.565,0.190,0.127,0.341,0.388,0.604,0.600,0.531,0.174
average_NDCG,0.799,0.726,0.209,0.200,0.420,0.274,0.734,0.828,0.612,0.190
average_M1,0.833,0.667,0.167,0.167,0.500,0.167,0.833,1.000,0.667,0.000
average_M3,1.000,1.000,0.167,0.333,0.500,0.167,0.833,1.000,0.667,0.000
average_M5,1.000,1.000,0.333,0.333,0.500,0.167,0.833,1.000,0.667,0.167
average_position,1.167,1.333,2.500,1.500,1.000,1.000,1.000,1.000,1.000,5.500
average_length (x of gs),0.928,1.228,0.967,0.780,0.689,0.883,0.433,0.600,0.639,2.067


Now let us combine the individual results of the apps.

In [10]:
combinedResults = {}
for label, keys in combineLabels.items():
    collectedResults = {}
    for key in keys:
        result = results[key]
        app = key.split('_')[2]
        collectedResults[app] = result.loc[:,app]
    combinedResults[label] = pd.DataFrame(
        collectedResults,
        index=[
            f"average_{x}" for x in [
                "precision", "recall", "f1-score", "NDCG", "M1", "M3", "M5", "position", "length (x of gs)"
            ]
        ]
        )

In [11]:
displayResults(combinedResults,printNumCases=False)

Results for experiment ignore_common_session_failed is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.482,0.389,0.023,0.341,0.513,0.257
average_recall,0.620,0.663,0.026,0.340,0.520,0.570
average_f1-score,0.542,0.490,0.024,0.340,0.516,0.354
average_NDCG,0.721,0.664,0.027,0.393,0.666,0.498
average_M1,0.612,0.469,0.021,0.326,0.636,0.271
average_M3,0.878,0.816,0.021,0.522,0.750,0.479
average_M5,0.898,0.837,0.043,0.522,0.795,0.625
average_position,1.432,1.810,2.500,1.542,1.371,2.765
average_length (x of gs),1.554,1.901,0.967,1.020,1.227,2.523


Results for experiment ignore_common_no_disease_found is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.482,0.389,0.271,0.392,0.461,0.257
average_recall,0.620,0.663,0.300,0.391,0.467,0.570
average_f1-score,0.542,0.490,0.285,0.391,0.464,0.354
average_NDCG,0.721,0.664,0.314,0.451,0.598,0.498
average_M1,0.612,0.469,0.250,0.375,0.571,0.271
average_M3,0.878,0.816,0.250,0.600,0.673,0.479
average_M5,0.898,0.837,0.500,0.600,0.714,0.625
average_position,1.432,1.810,2.500,1.542,1.371,2.765
average_length (x of gs),1.554,1.901,0.967,1.020,1.227,2.523


Results for experiment ignore_less-common_session_failed is


...

,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.374,0.387,0.0,0.197,0.223,0.159
average_recall,0.473,0.702,0.0,0.239,0.312,0.417
average_f1-score,0.418,0.499,NaN,0.216,0.260,0.230
average_NDCG,0.465,0.630,0.0,0.177,0.225,0.330
average_M1,0.321,0.464,0.0,0.045,0.107,0.179
average_M3,0.536,0.714,0.0,0.227,0.214,0.250
average_M5,0.643,0.786,0.0,0.227,0.214,0.393
average_position,2.111,1.818,0.0,2.000,2.000,2.364
average_length (x of gs),1.735,2.259,0.0,1.229,1.362,2.768


Results for experiment ignore_all-cases_session_failed is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.449,0.389,0.015,0.278,0.451,0.221
average_recall,0.574,0.677,0.017,0.290,0.495,0.514
average_f1-score,0.504,0.494,0.016,0.284,0.472,0.309
average_NDCG,0.636,0.652,0.017,0.305,0.557,0.436
average_M1,0.513,0.468,0.014,0.222,0.484,0.237
average_M3,0.763,0.779,0.014,0.403,0.609,0.395
average_M5,0.816,0.818,0.028,0.403,0.641,0.539
average_position,1.629,1.812,2.500,1.621,1.463,2.667
average_length (x of gs),1.618,2.031,0.967,1.094,1.270,2.615


Results for experiment ignore_all-cases_no_disease_found is


,Ada,Avey,Babylon,Buoy,K health,WebMD
average_precision,0.443,0.389,0.155,0.323,0.374,0.221
average_recall,0.567,0.677,0.171,0.337,0.411,0.514
average_f1-score,0.497,0.494,0.163,0.330,0.392,0.309
average_NDCG,0.628,0.652,0.179,0.354,0.463,0.436
average_M1,0.506,0.468,0.143,0.258,0.403,0.237
average_M3,0.753,0.779,0.143,0.468,0.506,0.395
average_M5,0.805,0.818,0.286,0.468,0.532,0.539
average_position,1.629,1.812,2.500,1.621,1.463,2.667
average_length (x of gs),1.618,2.031,0.967,1.094,1.270,2.615
